In [1]:
# ====================== 环境准备 ======================
# 1) 设定环境变量（必须在导入 matplotlib 之前执行）
import os
os.environ['QT_API'] = 'pyqt5'        # 指定使用 PyQt5 作为 Qt 绑定
os.environ['MPLBACKEND'] = 'QtAgg'    # 指定 Matplotlib 后端为 QtAgg（更推荐，替代 TkAgg）

# 2) 启动 Qt 事件循环（控制台模式下，让 Qt 窗口能实时响应）
%gui qt5

# 3) 检查 matplotlib backend
import matplotlib as mpl
mpl.rcParams.update({
    # 这里按系统常见字体给一串候选，存在则自动生效
    "font.sans-serif": ["Microsoft YaHei", "SimHei", "SimSun",
                        "Noto Sans CJK SC", "Source Han Sans SC",
                        "Arial Unicode MS", "DejaVu Sans"],
    "font.family": "sans-serif",
    "axes.unicode_minus": False,   # 负号用正常字符，避免被当作缺字形
})

print("backend (before pyplot):", mpl.get_backend())
# 如果不是 QtAgg，强制改为 QtAgg（注意：必须在导入 pyplot 前设置）
mpl.rcParams['backend'] = 'QtAgg'

# 4) 现在再导入 pyplot
import matplotlib.pyplot as plt
print("backend (after pyplot):", mpl.get_backend())



backend (before pyplot): QtAgg
backend (after pyplot): qtagg


In [2]:

# ====================== 导入依赖 ======================
import sys
# 避免反复执行时 Qt 类重复导入导致崩溃：如果已加载，先删除再导入
if 'draw.pyqt_draw.pyqt_main2' in sys.modules:
    del sys.modules['draw.pyqt_draw.pyqt_main2']

from PyQt5 import QtWidgets
import pyqtgraph as pg
from draw.pyqt_draw.pyqt_main2 import SatelliteViewer
import draw.read_snap_xml as read_snap_xml

# 配置 pyqtgraph：开启抗锯齿，关闭 OpenGL（更稳定）
pg.setConfigOptions(antialias=True)

In [9]:
# ====================== 基础参数 ======================
# 星座参数：每轨道卫星数 N，轨道平面数 P
N = 36
P = 18

# ====================== 读取数据 ======================
xml_file = r"E:\Data\station_visible_satellites_648_1d_real.xml"
start_ts = 0
# end_ts   = 86399
end_ts   = 1203
# 解析 XML 得到 group_data，结构：{time_step: {'groups': {...}}}
group_data = read_snap_xml.parse_xml_group_data(xml_file, start_ts, end_ts)

#下面是图变换的。
#

In [12]:
rev_group_data,offset = read_snap_xml.modify_group_data(group_data, N=36, groupid=4)

In [13]:

# ====================== 绘图初始化 ======================
# 1) QApplication 实例（全局唯一）
app = QtWidgets.QApplication.instance() or QtWidgets.QApplication([])

# 2) 确保 viewer 有全局引用，避免 GC 回收导致崩溃
if not hasattr(sys.modules[__name__], "_viewer_list"):
    _viewer_list = []

# 3) 创建并配置 viewer
viewer = SatelliteViewer(group_data)
viewer.setWindowTitle("Grouped Satellite Visibility - High Performance (PyQtGraph)")
viewer.resize(1200, 700)
# viewer.edges_by_step =
# viewer.pending_links_by_step =
viewer.show()



In [11]:
# 4) 保存全局引用,只要放入到这个容器里，就能持久存在
_viewer_list.append(viewer)

In [12]:

# ====================== 静态包络矩形 ======================
#可选
rects = {
    4: [(9, 15, 32, 35)],
    0: [(0, 5, 9, 13), (17, 17, 30, 32)],
}
# rects = {
#     4: (t1, t2),
#     0: (t3, t4),
# }

colors = {4: "deeppink", 0: "orange"}  # 可选

viewer.show_envelopes_static(
    rects_by_group=rects,
    expand=0.35,
    colors=colors,
    persist=True
)

In [ ]:


# ====================== 计算 Block 尺寸 ======================
import draw.basic_show.get_satellite_block_info as get_satellite_block_info


times = sorted(group_data.keys())   # 时间序列（所有 step）

# 下述代码是为了计算某个区域的块分布
groupid = 0  # 目标分组 ID

# 存储 Block1/Block2 的长宽随时间变化
widths1, heights1, widths2, heights2 = [], [], [], []
for t in times:
    sats4 = group_data[t]['groups'][groupid]
    # 返回：Block1 (长度, 宽度)，Block2 (长度, 宽度)
    (l1, w1), (l2, w2) = get_satellite_block_info.get_satellite_block_info(sats4, P, N)
    widths1.append(l1); heights1.append(w1)
    widths2.append(l2); heights2.append(w2)

# ====================== 可视化 ======================
# 创建上下两个子图：Block1 和 Block2
fig, (ax1, ax2) = plt.subplots(
    2, 1, figsize=(12, 8), sharex=True,
    gridspec_kw={'height_ratios': [1, 1]}
)
# -------- Block1 曲线 --------
ax1.plot(times, widths1,  label='Block1 Length', color='tab:blue')
ax1.plot(times, heights1, label='Block1 Width',  color='tab:orange')
ax1.set_ylabel('Block1 Size')
ax1.set_title('Block1（一般为主包络）长宽随时间变化')
ax1.legend()
ax1.grid(alpha=0.3)
# -------- Block2 线 --------
ax2.plot(times, widths2,  label='Block2 Length', color='tab:green')
ax2.plot(times, heights2, label='Block2 Width',  color='tab:red')
ax2.set_ylabel('Block2 Size')
ax2.set_title('Block2（有时不存在）长宽随时间变化')
ax2.legend()
ax2.grid(alpha=0.3)
# 公共横轴
plt.xlabel('Time Step')
# 自动调整布局，避免文字重叠
plt.tight_layout()
# 显示图形（独立 Qt 窗口，不阻塞控制台）
plt.show()


In [ ]:

# get the rectangular size of the group
# 同样是计算block尺寸
import draw.basic_functio.get_rectangular_size_interval as get_rectangular_size_interval
t1,t2=get_rectangular_size_interval.calc_envelope_for_group(group_data,[start_ts,end_ts],groupid,P,N)

In [13]:

##
# 生成无需 step 的边
edges = {}
for i in range(P - 1):
    for j in range(N):
        nownode = i * N + j
        next_node1 = (i + 1) * N + j
        edges.setdefault(nownode, set()).add(next_node1)
        upnodes = i * N + (j + 1) % N
        edges.setdefault(nownode, set()).add(upnodes)
        downnodes = i * N + (j - 1 + N) % N
        edges.setdefault(nownode, set()).add(downnodes)

In [ ]:


# 3) 下面是同构图设计，我们一般从同构图上设计出 motif，然后，再迁移到我们其他图的显示上去
import draw.pyqt_draw.pyqt_onetopology as pyqt_onetopology
viewer = pyqt_onetopology.Onetopology(rects)
viewer.setWindowTitle("Grouped Satellite Visibility - High Performance (PyQtGraph)")
viewer.resize(1200, 700)
motif_construct_edge = edges
viewer.edges_by_step = motif_construct_edge
viewer.show()
_viewer_list.append(viewer)

In [14]:

nodes = {}

import draw.basic_functio.motif as motif
# 下面就是motif图案填充，重复图案填充。里面有多种选项。
motif.write_distinct_motif(0, 17, 10, 35, P, N, nodes, option=0)
motif.write_distinct_motif(0, 17, 0, 9, P, N, nodes, option=1)

rev_inter_edge = motif.transform_nodes_2_adjacent(nodes,P,N)

In [15]:
all_rev_inter_edge = {}
for i in range(start_ts,end_ts):
    all_rev_inter_edge[i] = rev_inter_edge
# here the adj can be used for the qt5 to draw

In [16]:
import draw.basic_functio.revdata2rawdata as revdata2rawdata
# attention ,here  it just composed of the inter-link, intra_link hasn't benn conclued
raw_inter_edge = revdata2rawdata.revedge2rawedge(all_rev_inter_edge,offset)
import draw.basic_functio.conflict_link as conflict_link
time_2_build = 60
raw_edges_by_step, pending_links_by_step = conflict_link.get_no_conflict_link(raw_inter_edge,start_ts,end_ts,time_2_build,N,P)


44
33: (0,2) from None to (1, 1), (0,1) from (1, 1) to None, (0,11) from (1, 10) to (1, 11), (0,1) from (1, 1) to None, (1,2) from None to (2, 1), (1,1) from (2, 1) to None, (1,11) from (2, 10) to (2, 11), (1,1) from (2, 1) to None, (2,2) from None to (3, 1), (2,1) from (3, 1) to None, (2,11) from (3, 10) to (3, 11), (2,1) from (3, 1) to None, (3,2) from None to (4, 1), (3,1) from (4, 1) to None, (3,11) from (4, 10) to (4, 11), (3,1) from (4, 1) to None, (4,2) from None to (5, 1), (4,1) from (5, 1) to None, (4,11) from (5, 10) to (5, 11), (4,1) from (5, 1) to None, (5,2) from None to (6, 1), (5,1) from (6, 1) to None, (5,11) from (6, 10) to (6, 11), (5,1) from (6, 1) to None, (6,2) from None to (7, 1), (6,1) from (7, 1) to None, (6,11) from (7, 10) to (7, 11), (6,1) from (7, 1) to None, (7,2) from None to (8, 1), (7,1) from (8, 1) to None, (7,11) from (8, 10) to (8, 11), (7,1) from (8, 1) to None, (8,2) from None to (9, 1), (8,1) from (9, 1) to None, (8,11) from (9, 10) to (9, 11), (8,

In [17]:

# 3) 创建并配置 viewer
viewer = SatelliteViewer(group_data)
viewer.setWindowTitle("groupdata with rawedge")
viewer.resize(1200, 700)
viewer.edges_by_step =raw_edges_by_step
viewer.pending_links_by_step =pending_links_by_step
viewer.show()
